**Q**: is it ever possible to pass the Mizuno limit when the transfer matrix between the input and output is diagonal? I.e. for input-output transfer functions of the form,

$$
T_{uy}(\omega) = \prod_{j=0}^n \frac{\omega + \Delta_j - i\gamma_j}{\omega + \Delta_j + i\gamma_j}.
$$

We need to know whether or not a given realisation produced using the framework for a given transfer function is unique. For now we will assume that it is.

In [1]:
from simba import tf2rss, adiabatically_eliminate
import sympy
from sympy import I
from functools import reduce

omega = sympy.Symbol('omega')
s = sympy.Symbol('s')

def generate_tf(n, signs=None):
    """
    Generate n-dof lossless transfer function with the signs of the
    bandwidth of each mode being described by signs.
    
    e.g. generate_tf(2, [1, -1]) for transfer function for lossless
    coupled-cavity with second d.o.f. having negative dispersion.
    (Zero is treated as positive)
    
    If signs is not given assume all positive.
    """
    if n <= 0:
        raise ValueError("n must be > 0")
    if signs is None:
        signs = [1] * n
    if len(signs) != n:
        raise ValueError("signs must have length n")
        
    tfs = []
    
    for i in range(1, n + 1):
        Delta_i = sympy.Symbol(f'Delta_{i}', real=True)
        is_positive = signs[i - 1] >= 0
        gamma_i = sympy.Symbol(f'gamma_{i}', real=True, positive=is_positive)

        tfs.append((omega + Delta_i - I*gamma_i) / (omega + Delta_i + I*gamma_i))
    
    return reduce(lambda a, b: a * b, tfs).subs(omega, -I * s).simplify()

In [2]:
def get_all_tfs_to_internals(tf, num_dofs):
    """
    Get all transfer functions from the input to internal degrees
    of freedom.
    """

    split_network = tf2rss(tf).to_slh().split()
    tfm = split_network.tfm
    gamma, = split_network.aux_coupling_constants
    
    # special case for num_dofs == 1
    if num_dofs == 1:
        tfs = [tfm.open_loop('ain', 'a').simplify(),
               tfm.open_loop('ain', "a'").simplify(),
               tfm.open_loop('ain', 'conjugate(a)').simplify(),
               tfm.open_loop('ain', "conjugate(a')").simplify()]
    else:
        tfs = []

        for i in range(1, num_dofs + 1):
            tfs.append(tfm.open_loop('ain_1', f'a_{i}').simplify())
            tfs.append(tfm.open_loop('ain_1', f'conjugate(a_{i})').simplify())
            tfs.append(tfm.open_loop('ain_1', f'a_{i}\'').simplify())
            tfs.append(tfm.open_loop('ain_1', f'conjugate(a_{i}\')').simplify())
    
    tfs = list(map(lambda expr: adiabatically_eliminate(expr, gamma), tfs))
    return sympy.Matrix(tfs)
    

In [3]:
get_all_tfs_to_internals(generate_tf(1), 1)

Matrix([
[sqrt(2)*sqrt(gamma_1)/(I*Delta_1 + gamma_1 - s)],
[                                              0],
[                                              0],
[                                              0]])